# 总体目标：

由于事件驱动的计算架构是静态映射，需要先申请N个神经元
SNN模型：第一层n_1个神经元，第二层n_2个....

core 1 包含n_1和n_2个神经元（假设就是两个群），core 2和core 3包含n_3的神经元....

把这些神经元，群，core组成模型的拓扑结构


## step 1：
创建进程（process）,确定每个计算核心（core）上需要多少神经元

### step 1.1:
定义神经元这个类，这里以LIF为例

In [1]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort

class LIF(AbstractProcess):
    """Leaky-Integrate-and-Fire (LIF) neural Process.

    LIF dynamics abstracts to:
    u[t] = u[t-1] * (1-du) + a_in                              # neuron current
    v[t] = v[t-1] * (1-dv) + u[t] + bias_mant * 2 ** bias_exp  # neuron voltage
    s_out = v[t] > vth                                         # spike if threshold is exceeded
    v[t] = 0                                                   # reset at spike

    Parameters
    ----------
    du: Inverse of decay time-constant for current decay.
    dv: Inverse of decay time-constant for voltage decay.
    bias_mant: Mantissa part of neuron bias.
    bias_exp: Exponent part of neuron bias, if needed. Mostly for fixed point
              implementations. Unnecessary for floating point
              implementations. If specified, bias = bias_mant * 2**bias_exp.
    vth: Neuron threshold voltage, exceeding which, the neuron will spike.
    """
    def __init__(self, **kwargs):
        super().__init__()
        shape = kwargs.get("shape", (1,))
        du = kwargs.pop("du", 0)
        dv = kwargs.pop("dv", 0)
        bias_mant = kwargs.pop("bias_mant", 0)
        bias_exp = kwargs.pop("bias_exp", 0)
        vth = kwargs.pop("vth", 10)

        self.shape = shape
        self.a_in = InPort(shape=shape)
        self.s_out = OutPort(shape=shape)
        self.u = Var(shape=shape, init=0)
        self.v = Var(shape=shape, init=0)
        self.du = Var(shape=(1,), init=du)
        self.dv = Var(shape=(1,), init=dv)
        self.bias_mant = Var(shape=shape, init=bias_mant)
        self.bias_exp = Var(shape=shape, init=bias_exp)
        self.vth = Var(shape=(1,), init=vth)

## step 2: 
定义该process的具体行为。
这里用"@requires"确定所需的计算资源，

In [3]:
import numpy as np
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.model.py.model import PyLoihiProcessModel

@implements(proc=LIF, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyLifModel1(PyLoihiProcessModel):
    a_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float)
    s_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1)
    u: np.ndarray = LavaPyType(np.ndarray, float)
    v: np.ndarray = LavaPyType(np.ndarray, float)
    bias_mant: np.ndarray = LavaPyType(np.ndarray, float)
    bias_exp: np.ndarray = LavaPyType(np.ndarray, float)
    du: float = LavaPyType(float, float)
    dv: float = LavaPyType(float, float)
    vth: float = LavaPyType(float, float)

    def run_spk(self):
        a_in_data = self.a_in.recv()
        self.u[:] = self.u * (1 - self.du)
        self.u[:] += a_in_data
        bias = self.bias_mant * (2 ** self.bias_exp)
        self.v[:] = self.v * (1 - self.dv) + self.u + bias
        s_out = self.v >= self.vth
        self.v[s_out] = 0  # Reset voltage to 0
        self.s_out.send(
            
            
        )

## step 3: 
实例化，确定在之前的计算核心上需要几个神经元

In [4]:
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

n_neurons = 3
lif = LIF(shape=(3,), du=0, dv=0, bias_mant=3, vth=10)

## step 4: 执行
其中 RunSteps 类用于指定一个过程（Process）应该被执行多长时间，具体来说是以时间步长（time steps）为单位。时间步长（time steps）是计算模型和模拟环境中用于推进系统状态的最小时间间隔。它也被称为动力学速率。时间步长的大小会影响模拟的精度和效率。较小的时间步长可以提高精度，但会增加计算成本。较大的时间步长可以提高效率，但可能会降低精度。

利用实例测下来，我PC上时间耗费偏长

In [ ]:
run_cfg = Loihi1SimCfg()
lif.run(condition=RunSteps(num_steps=1), run_cfg=run_cfg)
print(lif.v.get())

下面我试着用小的例子测试

In [6]:
lif = LIF(shape=(1,))
lif.run(condition=RunSteps(num_steps=1), run_cfg=Loihi1SimCfg())
#时间依旧偏长

In [5]:
lif = LIF(shape=(1,))
# 设置时间步长为1毫秒
sim_cfg = Loihi1SimCfg(dt=0.001)

# 创建仿真过程并运行
lif.run(condition=RunSteps(num_steps=1), run_cfg=sim_cfg)

TypeError: __init__() got an unexpected keyword argument 'dt'

## CPU中模拟Loihi
Loihi1SimCfg 是一种运行配置（RunConfig），它指定了在模拟环境中使用单个CPU来模拟Loihi 1芯片的行为。这种配置使得开发者可以在没有实际Loihi硬件的情况下测试和开发与Loihi 1兼容的过程和模型。

因此下面的代码是使用Loihi1SimCfg()运行配置来在模拟Loihi 1环境中执行42个时间步。这意味着lif过程将会在CPU上模拟Loihi 1的行为，而不是在实际的Loihi 1硬件上运行。

In [1]:
from lava.magma.core.run_configs import Loihi1SimCfg

run_cfg = Loihi1SimCfg()
from lava.proc.lif.process import LIF
from lava.magma.core.run_conditions import RunSteps

# create a Process for a LIF neuron
lif = LIF(shape=(1,))

# execute that Process for 42 time steps in simulation
lif.run(condition=RunSteps(num_steps=42), run_cfg=Loihi1SimCfg())


## 神经元组成的群
设定每个群的神经元数量，数个群在一块计算资源上（core）上运行

In [5]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort


class Dense(AbstractProcess):
    """Dense connections between neurons.
    Realizes the following abstract behavior:
    a_out = W * s_in
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get("shape", (1, 1))
        self.s_in = InPort(shape=(shape[1],))
        self.a_out = OutPort(shape=(shape[0],))
        self.weights = Var(shape=shape, init=kwargs.pop("weights", 0))

In [6]:
import numpy as np
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg

# create processes
lif1 = LIF(shape=(1,))
dense = Dense(weights=np.eye(1))
lif2 = LIF(shape=(1,))

# connect the OutPort of lif1 to the InPort of dense
lif1.s_out.connect(dense.s_in)
# connect the OutPort of dense to the InPort of lif2
dense.a_out.connect(lif2.a_in)

# execute Process lif2 and all Processes connected to it (dense, lif1)
lif2.run(condition=RunSteps(num_steps=42), run_cfg=Loihi1SimCfg())

## 进程process之间的信号传输

In [1]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.ports.ports import InPort, OutPort
import numpy as np
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.resources import CPU
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

先定义演示用的抽象类

In [3]:
# Minimal process with an OutPort
class P1(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.out = OutPort(shape=shape)


# Minimal process with an InPort
class P2(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.inp = InPort(shape=shape)

In [4]:
# A minimal PyProcModel implementing P1
@implements(proc=P1, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyProcModelA(PyLoihiProcessModel):
    out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, int)

    def run_spk(self):
        data = np.array([1, 2])
        self.out.send(data)
        print("Sent output data of P1: {}".format(data))



# A minimal PyProcModel implementing P2
@implements(proc=P2, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyProcModelB(PyLoihiProcessModel):
    inp: PyInPort = LavaPyType(PyInPort.VEC_DENSE, int)

    def run_spk(self):
        in_data = self.inp.recv()
        print("Received input data for P2: {}".format(in_data))

实例化

In [5]:
sender = P1()
recv = P2()

# Connecting output port to an input port
sender.out.connect(recv.inp)

sender = P1()
recv = P2()

# ... or connecting an input port from an output port
recv.inp.connect_from(sender.out)

In [6]:
sender.run(RunSteps(num_steps=1), Loihi1SimCfg())
sender.stop()